In [1]:
import numpy as np
import warnings
import sys
import pandas as pd
import scipy
from scipy.io import arff
if not sys.warnoptions:
    warnings.simplefilter("ignore")
import pickle

In [2]:
def get_data(path, noise=False):
    data = np.load(path)
    if noise == True :
        data = data + np.random.normal(0, 0.001, data.shape)
    return data

In [3]:
X_train = get_data('delicious-train-features.pkl').astype(np.uint8)
y_train = get_data('delicious-train-labels.pkl').astype(np.int8)
X_test = get_data('delicious-test-features.pkl').astype(np.uint8)
y_test = get_data('delicious-test-labels.pkl').astype(np.int8)

In [4]:
print("Train_X: ",X_train.shape)
print("Train_Y: ",y_train.shape)
print("Test_X: ",X_test.shape)
print("Test_Y: ",y_test.shape)

Train_X:  (12920, 500)
Train_Y:  (12920, 983)
Test_X:  (3185, 500)
Test_Y:  (3185, 983)


## Accuracy Score


In [8]:
def accuracy_score(y_true, y_pred, normalize=True, sample_weight=None):
    '''
    Compute the Hamming score (a.k.a. label-based accuracy) for the multi-label case
    http://stackoverflow.com/q/32239577/395857
    '''
    acc_list = []
    for i in range(y_true.shape[0]):
        set_true = set( np.where(y_true[i])[0] )
        set_pred = set( np.where(y_pred[i])[0] )
        #print('\nset_true: {0}'.format(set_true))
        #print('set_pred: {0}'.format(set_pred))
        tmp_a = None
        if len(set_true) == 0 and len(set_pred) == 0:
            tmp_a = 1
        else:
            tmp_a = len(set_true.intersection(set_pred))/float( len(set_true.union(set_pred)) )
        #print('tmp_a: {0}'.format(tmp_a))
        acc_list.append(tmp_a)
    return np.mean(acc_list)

## MODEL 1: Chain Classifier of SVM

In [10]:
# using classifier chains
from skmultilearn.problem_transform import ClassifierChain
from sklearn.svm import SVC
from sklearn.metrics import hamming_loss, log_loss, f1_score

In [ ]:
# initialize classifier chains multi-label classifier
classifier = ClassifierChain(classifier = SVC())

# Training logistic regression model on train data
classifier.fit(X_train, y_train)

# predict
predictions = classifier.predict(X_test)

In [48]:
# accuracy
print("Accuracy = ",accuracy_score(y_test,predictions.toarray()))
print("Hamming loss = ",hamming_loss(y_test,predictions))
print("Log loss = ",log_loss(y_test,predictions.toarray()))
print("F1 score = ",f1_score(y_test,predictions.toarray(),average='macro'))

Accuracy =  0.659420289855
Hamming loss =  0.11580267558528429
Log loss =  12.7459829616
F1 score =  0.682575521853


## Model 2: Multi-lable Lazy Learning

In [12]:
from skmultilearn.adapt import MLkNN
from scipy.sparse import csr_matrix, lil_matrix
classifier_new = MLkNN(k=10)
# Note that this classifier can throw up errors when handling sparse matrices.
x_train = lil_matrix(X_train).toarray()
y_train = lil_matrix(y_train).toarray()
x_test = lil_matrix(X_test).toarray()
# train
classifier_new.fit(X_train, y_train)
# predict
predictions = classifier_new.predict(X_test)

In [27]:
# accuracy
print("Accuracy = ",accuracy_score(y_test,predictions.toarray()))
print("Hamming loss = ",hamming_loss(y_test,predictions))
print("Log loss = ",log_loss(y_test,predictions.toarray()))
print("F1 score = ",f1_score(y_test,predictions.toarray(),average='macro'))

Accuracy =  0.363628450805
Hamming loss =  0.05217192948153941
Log loss =  66.6688094837
F1 score =  0.101057668632


## Model 3: Classic ANN

In [5]:
import os
os.environ['CUDA_VISIBLE_DIVICES'] = '-1'
import keras
from keras.models import Sequential
from keras.layers import Dense,Dropout
from keras.backend.tensorflow_backend import clear_session
clear_session()
model = Sequential()
model.add(Dense(512,activation='relu',input_shape=(X_train.shape[1],),kernel_initializer='glorot_uniform'))
model.add(Dropout(0.4))
model.add(Dense(256,activation='relu',kernel_initializer='glorot_uniform'))
model.add(Dropout(0.4))
model.add(Dense(y_train.shape[1],activation='sigmoid',kernel_initializer='glorot_uniform'))
model.compile(loss='binary_crossentropy', optimizer='adagrad', metrics=['acc'])
model.summary()

Using TensorFlow backend.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 512)               256512    
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 256)               131328    
_________________________________________________________________
dropout_2 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 983)               252631    
Total params: 640,471
Trainable params: 640,471
Non-trainable params: 0
_________________________________________________________________


In [6]:
model.fit(x=X_train,y=y_train,epochs=5,batch_size=50)

Epoch 1/5
12920/12920 [==============================] - 26s 2ms/step - loss: 0.0821 - acc: 0.9788
Epoch 2/5
12920/12920 [==============================] - 2s 171us/step - loss: 0.0710 - acc: 0.9811
Epoch 3/5
12920/12920 [==============================] - 2s 172us/step - loss: 0.0687 - acc: 0.9813
Epoch 4/5
12920/12920 [==============================] - 2s 170us/step - loss: 0.0672 - acc: 0.9815
Epoch 5/5
12920/12920 [==============================] - 2s 170us/step - loss: 0.0661 - acc: 0.9816


In [11]:
#Predictions
predictions = model.predict(X_test)
predictions = np.array([list(np.round(x).astype(int)) for x in predictions])
# accuracy
print("Accuracy = ",accuracy_score(y_test,predictions))
print("Hamming loss = ",hamming_loss(y_test,predictions))
print("Log loss = ",log_loss(y_test,predictions))
print("F1 score = ",f1_score(y_test,predictions,average='macro'))

Accuracy =  0.104348369636
Hamming loss =  0.01811070777790731
Log loss =  420.005442442
F1 score =  0.0297101328517
